In [9]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [10]:
import nest_asyncio
nest_asyncio.apply()

Setup an agent over 3 papers

In [11]:
#urls = [
#    "https://openreview.net/pdf?id=VtmBAGCN7o",
#    "https://openreview.net/pdf?id=6PmJoRfdaK",
#    "https://openreview.net/pdf?id=hSyW5go0v8",
#]

papers = [
    "Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Finanza_a4fd7378c0.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Negocios_99e564b1a0.pdf",
]

In [12]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Finanza_a4fd7378c0.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Negocios_99e564b1a0.pdf


In [13]:
initial_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [14]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [15]:
len(initial_tools)

6

In [16]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    initial_tools, 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Tell me about the courses i will take in Business Administration "
    "and then tell me about the academic quality"
)

In [ ]:
response = agent.query("Give me a summary of both Self-RAG and LongLoRA")
print(str(response))

Setup an agent over 9 papers

In [18]:
papers = [
    "Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Finanza_a4fd7378c0.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Negocios_99e564b1a0.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Tec_DE_Info_4c05059c9f.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Ventas_b930c660ab.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Administracionde_Empresas_Turisticas_3f3c68ba10.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Ciencias_Politicasy_Administracion_Publica_d30ea21604.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Comunicacion_c27526bb56.pdf",
    "Utel_Peru_Fichas_Tecnicas_Carrera_Comunicacion_Digital_7bae61c167.pdf"
]

In [19]:
from utils import get_doc_tools
from pathlib import Path

paper_to_tools_dict = {}
for paper in papers:
    print(f"Getting tools for paper: {paper}")
    vector_tool, summary_tool = get_doc_tools(paper, Path(paper).stem)
    paper_to_tools_dict[paper] = [vector_tool, summary_tool]

Getting tools for paper: Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Finanza_a4fd7378c0.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Negocios_99e564b1a0.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Tec_DE_Info_4c05059c9f.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Admin_Ventas_b930c660ab.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Administracionde_Empresas_Turisticas_3f3c68ba10.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Ciencias_Politicasy_Administracion_Publica_d30ea21604.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Comunicacion_c27526bb56.pdf
Getting tools for paper: Utel_Peru_Fichas_Tecnicas_Carrera_Comunicacion_Digital_7bae61c167.pdf


Extend the Agent with Tool Retrieval

In [20]:
all_tools = [t for paper in papers for t in paper_to_tools_dict[paper]]

In [21]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [22]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

In [23]:
tools = obj_retriever.retrieve(
    "Tell me about the eval dataset used in MetaGPT and SWE-Bench"
)

In [26]:
tools[0].metadata

ToolMetadata(description='Use ONLY IF you want to get a holistic summary of MetaGPT. Do NOT use if you have specific questions over MetaGPT.', name='summary_tool_Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380', fn_schema=<class 'llama_index.core.tools.types.DefaultToolFnSchema'>, return_direct=False)

In [27]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    system_prompt=""" \
You are an agent designed to answer queries over a set of given papers.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True
)
agent = AgentRunner(agent_worker)

In [ ]:
response = agent.query(
    "Tell me about the courses i will take in Business Administration "
    "and then tell me about the academic quality"
)
print(str(response))

In [ ]:
response = agent.query(
    "Compare and contrast the LoRA papers (LongLoRA, LoftQ). "
    "Analyze the approach in each paper first. "
)